In [ ]:
# Imports

In [ ]:
# Parse existing SOM
from SOMToolBoxParser import SOMToolBoxParser
input_vectors = SOMToolBoxParser("datasets/iris/iris.vec").read_weight_file()
weight_vectors = SOMToolBoxParser("datasets/iris/iris.wgt.gz").read_weight_file()

# Metro Map Implementation

## Pseudocode

### 1. Read calculated SOM

Done, see parse existing SOM

output: dict

### 2.a Component Planes (M)

For each unit, take the weight vector of the corresponding attribute

output: 2d matrix
output: replace weight_vectors (reshape)
```
for each component:
[
    []
    []
    []
]
```

### 2.b Discretization (M)

Input parameter `n`: number of bins

Generate discretized SOM

output
for each component:
represent to which bin the unit belongs

### 3. Component Lines (Fi)

For each bin, calculate the center of gravity
Save min and max of centers (low, high)

### 4. Visualization (Fi)

Connect the centers with lines

### 5. Aggregation (FL)

distance-based

Take pair wise lines,
    calculate distance between two lines:
        same_idx_dist <- sum of distances between pairs of centers of same indices
        inv_idx_dist <- sum of distances between paris of centers of inverted indices
        dist <- min(same_idx_dist, inv_idx_dist)

use Ward's clustering (https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ward.html)

### 6. Selection

See Algorithm 1 (NEU07_wsom paper)
select only components with a small amount of regions per bin

### 7. Snapping

See Algorithm 1 (https://github.com/yozoon/SOM-MetroMap/blob/main/implementation_report.pdf)

See implementation of template method `_find_snapped_line`

## References



In [ ]:
# Metro Map Implementation

In [ ]:
# Metro Map Visualization